#### ***Step 1: Fit Separate Cox Models with Interaction Terms***

##### ***Trimmed Dataset***

In [3]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from sklearn.preprocessing import StandardScaler

# Load data
data_trimmed = pd.read_csv("D:/prethesis/data/antibiotics_time_to_event_trimmed.csv")

# Define confounders and covariates (excluding first_careunit due to PH violations)
covariates = ['has_antibiotic', 'age', 'White blood cell count_mean', 'Temperature_mean',
              'Oxygen saturation_mean', 'Creatinine_mean', 'Respiratory rate_mean',
              'admission_type_EMERGENCY', 'admission_type_URGENT', 'sepsis_icd9_1']

confounders = ['age', 'White blood cell count_mean', 'Temperature_mean',
               'Oxygen saturation_mean', 'Creatinine_mean', 'Respiratory rate_mean',
               'admission_type', 'sepsis_icd9']

# Prepare data
X = data_trimmed[confounders].copy()
numerical_vars = ['age', 'White blood cell count_mean', 'Temperature_mean',
                 'Oxygen saturation_mean', 'Creatinine_mean', 'Respiratory rate_mean']
categorical_vars = ['admission_type', 'sepsis_icd9']
scaler = StandardScaler()
X[numerical_vars] = scaler.fit_transform(X[numerical_vars])
X = pd.get_dummies(X, columns=categorical_vars, drop_first=True)

# Convert sepsis_icd9_1 to integer (0 and 1) to avoid issues with boolean type
X['sepsis_icd9_1'] = X['sepsis_icd9_1'].astype(int)

# Add interaction term
X['age_sepsis_interaction'] = X['age'] * X['sepsis_icd9_1']
X = X[[c for c in X.columns if c in covariates or c == 'age_sepsis_interaction']]
A = data_trimmed['has_antibiotic']
T = data_trimmed['time']
D = data_trimmed['status']

# Treated (has_antibiotic = 1)
data_treated = X[A == 1].copy()
data_treated['time'] = T[A == 1]
data_treated['status'] = D[A == 1]

# Check distribution of sepsis_icd9_1 in treated group
print("Distribution of sepsis_icd9_1 in treated group:")
print(data_treated['sepsis_icd9_1'].value_counts())

# Reset index to avoid indexing issues
data_treated = data_treated.reset_index(drop=True)

# Fit Cox model for treated group
try:
    cox_treated = CoxPHFitter(penalizer=0.5, strata=['sepsis_icd9_1'])
    cox_treated.fit(data_treated, duration_col='time', event_col='status')
    print("\nCox Model (Treated - Trimmed Dataset, with stratification):")
    cox_treated.print_summary()
except Exception as e:
    print(f"Stratification failed for treated group: {e}")
    # Fallback: Fit without stratification
    cox_treated = CoxPHFitter(penalizer=0.5)
    cox_treated.fit(data_treated, duration_col='time', event_col='status')
    print("\nCox Model (Treated - Trimmed Dataset, without stratification):")
    cox_treated.print_summary()

# Control (has_antibiotic = 0)
data_control = X[A == 0].copy()
data_control['time'] = T[A == 0]
data_control['status'] = D[A == 0]

# Check distribution of sepsis_icd9_1 in control group
print("\nDistribution of sepsis_icd9_1 in control group:")
print(data_control['sepsis_icd9_1'].value_counts())

# Reset index to avoid indexing issues
data_control = data_control.reset_index(drop=True)

# Fit Cox model for control group
try:
    cox_control = CoxPHFitter(penalizer=0.5, strata=['sepsis_icd9_1'])
    cox_control.fit(data_control, duration_col='time', event_col='status')
    print("\nCox Model (Control - Trimmed Dataset, with stratification):")
    cox_control.print_summary()
except Exception as e:
    print(f"Stratification failed for control group: {e}")
    # Fallback: Fit without stratification
    cox_control = CoxPHFitter(penalizer=0.5)
    cox_control.fit(data_control, duration_col='time', event_col='status')
    print("\nCox Model (Control - Trimmed Dataset, without stratification):")
    cox_control.print_summary()

Distribution of sepsis_icd9_1 in treated group:
sepsis_icd9_1
0    4305
1    1217
Name: count, dtype: int64

Cox Model (Treated - Trimmed Dataset, with stratification):


model,lifelines.CoxPHFitter
duration col,'time'
event col,'status'
penalizer,0.5
l1 ratio,0.0
strata,sepsis_icd9_1
baseline estimation,breslow
number of observations,5522
number of events observed,968
partial log-likelihood,-6719.87
time fit was run,2025-04-23 15:50:57 UTC



Distribution of sepsis_icd9_1 in control group:
sepsis_icd9_1
0    25950
1     1718
Name: count, dtype: int64

Cox Model (Control - Trimmed Dataset, with stratification):


model,lifelines.CoxPHFitter
duration col,'time'
event col,'status'
penalizer,0.5
l1 ratio,0.0
strata,sepsis_icd9_1
baseline estimation,breslow
number of observations,27668
number of events observed,3120
partial log-likelihood,-27663.39
time fit was run,2025-04-23 15:50:57 UTC


#### ***Step 2: Predict Counterfactual Outcomes***

In [4]:
# Predict survival curves for all patients under both scenarios
survival_treated = cox_treated.predict_survival_function(X)
survival_control = cox_control.predict_survival_function(X)

# Ensure time points are aligned
print("Time points for treated model:", survival_treated.index[:5])
print("Time points for control model:", survival_control.index[:5])

Time points for treated model: Index([0.0, 1.0, 2.0, 3.0, 4.0], dtype='float64')
Time points for control model: Index([0.0, 1.0, 2.0, 3.0, 4.0], dtype='float64')


#### ***Step 3: Calculate Individual RMSTs with Fine Time Increments***

In [6]:
from scipy.integrate import trapezoid 
import numpy as np

# Define time points with fine increments (0 to 30 days, 0.1-day increments)
tau = 30
time_points = np.arange(0, tau + 0.1, 0.1)

# Interpolate survival probabilities at fine time points
survival_treated_interp = np.array([np.interp(time_points, survival_treated.index, survival_treated.iloc[:, i])
                                    for i in range(survival_treated.shape[1])]).T
survival_control_interp = np.array([np.interp(time_points, survival_control.index, survival_control.iloc[:, i])
                                    for i in range(survival_control.shape[1])]).T

# Calculate RMST by integrating survival curves (area under the curve)
rmst_treated = trapezoid(survival_treated_interp, time_points, axis=0)  # Updated function
rmst_control = trapezoid(survival_control_interp, time_points, axis=0)  # Updated function

print("Sample RMST (Treated, first 5 patients):", rmst_treated[:5])
print("Sample RMST (Control, first 5 patients):", rmst_control[:5])

Sample RMST (Treated, first 5 patients): [23.17686975 22.78465281 23.45668828 23.40037178 23.63261911]
Sample RMST (Control, first 5 patients): [25.77579012 26.0743416  25.994471   25.85168784 25.93506889]


#### ***Step 4:Compute ATE***  

In [7]:
ate_trimmed = np.mean(rmst_treated - rmst_control)
print(f"G-formula ATE (Trimmed Dataset): {ate_trimmed:.2f} days")

G-formula ATE (Trimmed Dataset): -2.32 days


In [8]:
# Stratify by propensity score ranges
data_trimmed['prop_score_bin'] = pd.cut(data_trimmed['prop_score'], bins=[0, 0.25, 0.50, 1.0], labels=['Low (<0.25)', 'Medium (0.25-0.50)', 'High (>0.50)'])

# Compute ATE for each propensity score bin
for bin_name in data_trimmed['prop_score_bin'].unique():
    mask = data_trimmed['prop_score_bin'] == bin_name
    ate_bin = np.mean(rmst_treated[mask] - rmst_control[mask])
    print(f"ATE for {bin_name} propensity scores (Trimmed Dataset): {ate_bin:.2f} days (n={mask.sum()})")

ATE for Medium (0.25-0.50) propensity scores (Trimmed Dataset): -2.32 days (n=3079)
ATE for Low (<0.25) propensity scores (Trimmed Dataset): -2.32 days (n=29744)
ATE for High (>0.50) propensity scores (Trimmed Dataset): -2.37 days (n=367)


##### ***Matched Dataset*** 

In [9]:
# Load matched dataset
data_matched = pd.read_csv("D:/prethesis/data/antibiotics_time_to_event_matched.csv")

# Step 1*: Fit Separate Cox Models with Interaction Terms (Matched Dataset)
X_matched = data_matched[confounders].copy()
X_matched[numerical_vars] = scaler.fit_transform(X_matched[numerical_vars])
X_matched = pd.get_dummies(X_matched, columns=categorical_vars, drop_first=True)
X_matched['sepsis_icd9_1'] = X_matched['sepsis_icd9_1'].astype(int)
X_matched['age_sepsis_interaction'] = X_matched['age'] * X_matched['sepsis_icd9_1']
X_matched = X_matched[[c for c in X_matched.columns if c in covariates or c == 'age_sepsis_interaction']]
A_matched = data_matched['has_antibiotic']
T_matched = data_matched['time']
D_matched = data_matched['status']

# Treated (has_antibiotic = 1)
data_treated_matched = X_matched[A_matched == 1].copy()
data_treated_matched['time'] = T_matched[A_matched == 1]
data_treated_matched['status'] = D_matched[A_matched == 1]
data_treated_matched = data_treated_matched.reset_index(drop=True)

# Fit Cox model for treated group (matched dataset)
try:
    cox_treated_matched = CoxPHFitter(penalizer=0.5, strata=['sepsis_icd9_1'])
    cox_treated_matched.fit(data_treated_matched, duration_col='time', event_col='status')
    print("\nCox Model (Treated - Matched Dataset, with stratification):")
    cox_treated_matched.print_summary()
except Exception as e:
    print(f"Stratification failed for treated group (matched dataset): {e}")
    cox_treated_matched = CoxPHFitter(penalizer=0.5)
    cox_treated_matched.fit(data_treated_matched, duration_col='time', event_col='status')
    print("\nCox Model (Treated - Matched Dataset, without stratification):")
    cox_treated_matched.print_summary()

# Control (has_antibiotic = 0)
data_control_matched = X_matched[A_matched == 0].copy()
data_control_matched['time'] = T_matched[A_matched == 0]
data_control_matched['status'] = D_matched[A_matched == 0]
data_control_matched = data_control_matched.reset_index(drop=True)

# Fit Cox model for control group (matched dataset)
try:
    cox_control_matched = CoxPHFitter(penalizer=0.5, strata=['sepsis_icd9_1'])
    cox_control_matched.fit(data_control_matched, duration_col='time', event_col='status')
    print("\nCox Model (Control - Matched Dataset, with stratification):")
    cox_control_matched.print_summary()
except Exception as e:
    print(f"Stratification failed for control group (matched dataset): {e}")
    cox_control_matched = CoxPHFitter(penalizer=0.5)
    cox_control_matched.fit(data_control_matched, duration_col='time', event_col='status')
    print("\nCox Model (Control - Matched Dataset, without stratification):")
    cox_control_matched.print_summary()

# Step 2*: Predict Counterfactual Outcomes (Matched Dataset)
survival_treated_matched = cox_treated_matched.predict_survival_function(X_matched)
survival_control_matched = cox_control_matched.predict_survival_function(X_matched)
print("Time points for treated model (matched):", survival_treated_matched.index[:5])
print("Time points for control model (matched):", survival_control_matched.index[:5])

# Step 3*: Calculate Individual RMSTs (Matched Dataset)
survival_treated_matched_interp = np.array([np.interp(time_points, survival_treated_matched.index, survival_treated_matched.iloc[:, i])
                                            for i in range(survival_treated_matched.shape[1])]).T
survival_control_matched_interp = np.array([np.interp(time_points, survival_control_matched.index, survival_control_matched.iloc[:, i])
                                            for i in range(survival_control_matched.shape[1])]).T

rmst_treated_matched = trapezoid(survival_treated_matched_interp, time_points, axis=0)  # Updated function
rmst_control_matched = trapezoid(survival_control_matched_interp, time_points, axis=0)  # Updated function

print("Sample RMST (Treated, Matched, first 5 patients):", rmst_treated_matched[:5])
print("Sample RMST (Control, Matched, first 5 patients):", rmst_control_matched[:5])

# Step 4*: Compute ATE (Matched Dataset)
ate_matched = np.mean(rmst_treated_matched - rmst_control_matched)
print(f"G-formula ATE (Matched Dataset): {ate_matched:.2f} days")


Cox Model (Treated - Matched Dataset, with stratification):


model,lifelines.CoxPHFitter
duration col,'time'
event col,'status'
penalizer,0.5
l1 ratio,0.0
strata,sepsis_icd9_1
baseline estimation,breslow
number of observations,5522
number of events observed,968
partial log-likelihood,-6719.87
time fit was run,2025-04-23 15:57:57 UTC



Cox Model (Control - Matched Dataset, with stratification):


model,lifelines.CoxPHFitter
duration col,'time'
event col,'status'
penalizer,0.5
l1 ratio,0.0
strata,sepsis_icd9_1
baseline estimation,breslow
number of observations,5522
number of events observed,915
partial log-likelihood,-6477.48
time fit was run,2025-04-23 15:57:57 UTC


Time points for treated model (matched): Index([0.0, 1.0, 2.0, 3.0, 4.0], dtype='float64')
Time points for control model (matched): Index([0.0, 1.0, 2.0, 3.0, 4.0], dtype='float64')
Sample RMST (Treated, Matched, first 5 patients): [23.48858545 22.43911708 21.27741275 22.7295859  24.44911744]
Sample RMST (Control, Matched, first 5 patients): [25.4474311  24.3992941  23.10934224 25.15307165 26.17272883]
G-formula ATE (Matched Dataset): -1.65 days
